# 👋 Olá, estou muito feliz em vê-lo aqui. 
Presumo que você está procurando uma maneira fácil de criar seu avatar com uma inteligência artificial. Bem, é para isso que estou aqui, vou lhe guiar nessa jornada.

Abaixo, está o passo a passo de como você pode criar as imagens, siga-as, e caso ocorra algum problema, você tenha uma dúvida, ou ainda uma melhoria, siga para [esse repositório do Github](https://github.com/Dpbm/dreambooth-tutorial) e abra uma `ISSUE`, assim que possível a comunidade irá te ajudar!

Pronto para começar?

# 1 - HuggingFace

![HuggingFace ícone](./assets/hugging-face-image.svg)

O huggingFace, é uma plataforma, da qual pessoas engajadas no universo de inteligência artificial podem compartilhar pesquisas, [modelos](https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial), etc.\
Além de possibilitar pessoas que não são da área, a utilizarem essas IAs de uma forma fácil através de uma [API](https://pt.wikipedia.org/wiki/Interface_de_programa%C3%A7%C3%A3o_de_aplica%C3%A7%C3%B5es).

Aqui utilizaremos para nos comunicarmos com um modelo chamado [Stable Diffusion](https://huggingface.co/runwayml/stable-diffusion-v1-5).

1. Para começar, entre em [huggingface.co](https://huggingface.co) e [crie uma conta](https://huggingface.co/join), caso você não possua uma;
2. Em seguida, o HuggingFace irá lhe enviar um e-mail para verificar sua conta, entre nele e clique no link que lhe enviaram (lembre de olhar também a caixa de spam);
3. Com sua conta verificada, [clique aqui](https://huggingface.co/settings/tokens);
4. Clique em `New Token`(ou nova token, dependendo do idioma que você está utilizando);
5. O seguinte campo abrirá em sua tela:

![campo para criar o token](./assets/token_field.png)

Em `name` coloque um nome para sua token, recomendo colocar um nome sem caracteres especiais (como ponto, vírgula, espaço, etc.) e sem acentos, isso pode gerar erros mais tarde.\
Em `role` mude para `write` (ou escrita, dependendo do seu idioma do navegador).

6. Após criada a token, `anote o nome da token em algum local` e depois clique em `copiar` e `também à anote a token gerada em algum local`

![copiar token](./assets/copy_token.png)
 